<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Практическое-упражнение:" data-toc-modified-id="Практическое-упражнение:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Практическое упражнение:</a></span></li><li><span><a href="#Обучение-моделей:" data-toc-modified-id="Обучение-моделей:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей:</a></span></li></ul></div>

In [1]:
import numpy as np

In [2]:
class Node:
    """Node class will hold information about one node in the decision tree."""

    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        """
        Initialization of decision node.

        :param int feature_index: Index of feature by which data was split
        :param float threshold: Threshold value by which data was split
        :param Node left: Left child node
        :param Node right: Right child node
        :param float info_gain: Information gain of the split
        :param float value: Value if the node is a leaf in the tree
        """

        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value


class DecisionTree:
    """Decision tree class for binary classification tasks."""

    def __init__(self, min_samples_split=2, max_depth=2):
        """
        Initialization of decision tree classifier.

        :param int min_samples_split: The minimum number of samples required to split an internal node
        :param int max_depth: The maximum depth of the tree
        """

        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    @staticmethod
    def entropy(y):
        """
        Calculate entropy of the labels.

        :param ndarray y: Target values
        :return: entropy
        :rtype: float
        """

        classes = np.unique(y)
        entropy = 0
        for cls in classes:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def best_split(self, X, y):
        """
        Find best feature and value for a split. Greedy algorithm.

        :param ndarray X: Feature dataset
        :param ndarray y: Target values
        :return: node information gain, feature index and threshold
        :rtype: tuple
        """

        best_split = {}
        max_info_gain = -1
        n_rows, n_cols = X.shape
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            for threshold in np.unique(X_curr):
                gain = self.information_gain(y, X_curr, threshold)
                if gain > max_info_gain:
                    max_info_gain = gain
                    best_split = {"feature_index": f_idx, "threshold": threshold, "info_gain": gain}
        return best_split

    def information_gain(self, y, X_col, split_thresh):
        """
        Compute information gain of a split.

        :param ndarray y: Target values
        :param ndarray X_col: Feature column values
        :param float split_thresh: Value to split the feature column
        :return: information gain of the split
        :rtype: float
        """

        parent_entropy = self.entropy(y)

        # Generate split
        left_idx = np.where(X_col < split_thresh)
        right_idx = np.where(X_col >= split_thresh)

        if len(left_idx[0]) == 0 or len(right_idx[0]) == 0:  # Skip if split not feasible
            return -1

        # Compute children entropy
        n = len(y)
        n_l, n_r = len(left_idx[0]), len(right_idx[0])
        e_l, e_r = self.entropy(y[left_idx]), self.entropy(y[right_idx])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # Compute information gain
        ig = parent_entropy - child_entropy
        return ig

    def build_tree(self, X, y, curr_depth=0):
        """
        Recursive method to create the decision tree.

        :param ndarray X: Feature dataset
        :param ndarray y: Target values
        :param int curr_depth: Current depth level
        :return: constructed node
        :rtype: Node
        """

        n_rows, n_cols = X.shape
        n_labels = len(np.unique(y))

        # Stop criteria
        if n_rows >= self.min_samples_split and n_labels > 1 and curr_depth <= self.max_depth:
            best = self.best_split(X, y)
            if best["info_gain"] > 0:
                left_subtree = self.build_tree(X[X[:, best["feature_index"]] < best["threshold"], :], y[X[:, best["feature_index"]] < best["threshold"]], curr_depth + 1)
                right_subtree = self.build_tree(X[X[:, best["feature_index"]] >= best["threshold"], :], y[X[:, best["feature_index"]] >= best["threshold"]], curr_depth + 1)
                return Node(best["feature_index"], best["threshold"], left_subtree, right_subtree, best["info_gain"])
        
        leaf_value = self.majority_vote(y)
        return Node(value=leaf_value)

    @staticmethod
    def majority_vote(y):
        """
        Return the majority voted class.

        :param ndarray y: Target values
        :return: majority class
        :rtype: int
        """

        most_common = np.argmax(np.bincount(y))
        return most_common

    def print_tree(self, tree=None, indent=" "):
        """
        Recursive print of decision tree structure.

        :param Node tree: Tree object
        :param str indent: Indentation string
        """

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)
        else:
            print(f'X_{tree.feature_index} <= {tree.threshold}')
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, y):
        """
        Build decision tree classifier.

        :param ndarray X: Feature dataset
        :param ndarray y: Target values
        """

        self.root = self.build_tree(X, y)

    def predict(self, X):
        """
        Make predictions of target values with the fitted decision tree.

        :param ndarray X: Feature dataset
        :return: predicted values
        :rtype: ndarray
        """

        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        """
        Auxiliary method to make predictions.

        :param list inputs: Single data instance
        :return: predicted class
        :rtype: int
        """

        node = self.root
        while node.value is None:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value


In [3]:
from sklearn.datasets import load_iris

data = load_iris()
X, y = data.data, data.target

tree = DecisionTree(max_depth=3)
tree.fit(X, y)

print(tree.predict([[0, 0, 5, 1.5]]))  # пример прогноза

tree.print_tree()  # вывод структуры дерева


[2]
X_2 <= 3.0
 left:0
 right:X_3 <= 1.8
  left:X_2 <= 5.0
    left:X_3 <= 1.7
        left:1
        right:2
    right:X_3 <= 1.6
        left:2
        right:1
  right:X_2 <= 4.9
    left:X_0 <= 6.0
        left:1
        right:2
    right:2


Суть ансамблевых методов заключается в создании набора простых моделей, которые, работая вместе, обеспечивают более высокую производительность, чем каждая из моделей по отдельности. Другими словами, группа слабых моделей, объединенных вместе, может дать в совокупности более сильный и точный прогноз. Этот подход основан на поговорке "Много глаз видят больше".

Важным преимуществом ансамблевых методов является их способность уменьшить переобучение, которое является одной из основных проблем в машинном обучении. Переобученная модель "слишком хорошо" подходит к тренировочным данным, но плохо обобщает новые, неизвестные данные. Ансамблевые методы могут уменьшить вероятность переобучения, поскольку они усредняют предсказания нескольких моделей, что помогает сгладить выбросы и уменьшить дисперсию.

Еще одно важное преимущество ансамблевых методов - это повышение точности прогнозирования. Используя несколько моделей вместо одной, мы получаем возможность учесть больше нюансов в данных и сделать более точные прогнозы.

Ансамблевые методы можно разделить на несколько основных категорий, среди которых самыми распространенными являются бэггинг и бустинг.

Бэггинг, что является сокращением от "bootstrap aggregating", заключается в генерации нескольких подвыборок из исходного набора данных с заменой, обучении отдельной модели для каждой из подвыборок, а затем усреднении их прогнозов. Примером алгоритма, использующего бэггинг, является Random Forest.

Бустинг, в свою очередь, представляет собой итеративный процесс, где каждая следующая модель стремится исправить ошибки предыдущей. Градиентный бустинг - это одна из популярных техник бустинга, которая оптимизирует функцию потерь, пытаясь сделать больший упор на неправильно классифицированных примерах на предыдущих итерациях.

Бэггинг, или bootstrap aggregating, это метод, который генерирует несколько подвыборок из исходного набора данных с заменой и обучает модель на каждом из этих подмножеств. Затем берутся прогнозы каждой обученной модели и происходит их агрегация, или усреднение. Это означает, что для задачи регрессии мы просто усредним прогнозы, а для задачи классификации мы выберем класс, который получил большинство голосов.

Основное преимущество бэггинга заключается в уменьшении дисперсии модели. Дисперсия описывает, насколько предсказания модели изменяются для данного наблюдаемого значения. В контексте машинного обучения высокая дисперсия может указывать на переобучение, когда модель слишком хорошо подходит к обучающим данным и плохо обобщает на новые данные. Бэггинг уменьшает дисперсию, генерируя большое количество подмножеств данных и обучая модель на каждом из них. Это позволяет учесть различные аспекты данных и сгладить выбросы.

Давайте рассмотрим пример применения бэггинга: алгоритм случайного леса. Random Forest - это мощный алгоритм машинного обучения, который использует бэггинг в сочетании с деревьями решений. Для каждого дерева в лесу создается подвыборка обучающих данных с заменой, а затем дерево обучается на этом подмножестве. Важно отметить, что при каждом разделении дерева используется случайный поднабор признаков, что добавляет дополнительную стохастичность в модель и помогает уменьшить корреляцию между деревьями.

Давайте перейдем к примерам использования бэггинга. Этот метод широко используется во многих областях, где требуется машинное обучение.

Прогнозирование в области финансов: Бэггинг может быть использован для прогнозирования финансовых рынков. В этом случае несколько моделей обучаются на различных подмножествах финансовых данных и их прогнозы агрегируются для получения конечного прогноза.

Медицинская диагностика: Бэггинг также используется в медицинской диагностике, например, для обнаружения заболеваний на основе медицинских изображений. Несколько моделей обучаются на различных подмножествах данных и их прогнозы агрегируются для получения конечного диагноза.

Биология и экология: В этих областях бэггинг используется для классификации и идентификации видов на основе их характеристик.

Следует отметить, что бэггинг часто используется в сочетании с алгоритмами машинного обучения, основанными на деревьях решений. Деревья решений являются хорошим выбором для бэггинга, поскольку они могут обучаться на разных подмножествах данных и могут создавать сложные модели.

Теперь давайте рассмотрим, как можно применить бэггинг с использованием библиотеки Scikit-Learn. Мы будем использовать BaggingClassifier, который предоставляет все необходимые инструменты для реализации бэггинга.

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Создание искусственного набора данных
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация базового классификатора
base_cls = KNeighborsClassifier()

# Инициализация BaggingClassifier с KNeighborsClassifier в качестве базового классификатора
bag_cls = BaggingClassifier(base_estimator=base_cls, n_estimators=10, random_state=42)

# Обучение BaggingClassifier
bag_cls.fit(X_train, y_train)

# Оценка BaggingClassifier
print("BaggingClassifier accuracy: ", bag_cls.score(X_test, y_test))


BaggingClassifier accuracy:  0.8733333333333333


C:\Users\Sidromnik\.conda\envs\TeachMeSkills\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


"Бустинг" является другим важным ансамблевым методом. Он работает последовательно, то есть каждая последующая модель в ансамбле стремится исправить ошибки предыдущей.

Модели создаются последовательно, где каждая следующая модель учится исправлять ошибки предыдущей. Это происходит за счет концентрации на тех примерах обучающего набора, которые были классифицированы неправильно или которые представляют больший вызов для модели.

Основное преимущество бустинга - это его способность увеличивать точность модели. Однако это также может привести к риску переобучения, если не ограничить сложность базовых моделей или не использовать регуляризацию.

Примером алгоритма бустинга является градиентный бустинг, который обычно использует деревья решений в качестве базовых моделей.

Бустинг широко используется в многих приложениях, где требуется максимальная точность прогнозов.

Прогнозирование в области финансов: Бустинг используется для прогнозирования цен акций, валютных курсов и других финансовых показателей.

Медицинская диагностика: Бустинг может быть использован для обнаружения заболеваний на основе медицинских изображений или данных пациентов.

Обработка естественного языка (NLP): Бустинг также используется в NLP для классификации текстов и анализа настроений.

Теперь давайте рассмотрим, как можно применить бустинг с использованием библиотеки Scikit-Learn. Мы будем использовать GradientBoostingClassifier.

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Создание искусственного набора данных
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация GradientBoostingClassifier
gb_cls = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Обучение GradientBoostingClassifier
gb_cls.fit(X_train, y_train)

# Оценка GradientBoostingClassifier
print("GradientBoostingClassifier accuracy: ", gb_cls.score(X_test, y_test))


GradientBoostingClassifier accuracy:  0.8866666666666667


## Практическое упражнение:
Реализация бэггинга и бустинга с использованием Python и Scikit-Learn

Задача: В данном упражнении мы будем использовать классификаторы Bagging и Gradient Boosting для решения задачи классификации и сравним их результаты с результатами, полученными от отдельных деревьев решений.

1. Подготовка данных:

Для упражнения мы будем использовать датасет Iris из библиотеки sklearn. Загрузим датасет и разделим его на обучающую и тестовую выборки:

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Обучение моделей:

Сначала обучим одиночное дерево решений:

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
print("Decision Tree accuracy: ", tree.score(X_test, y_test))


Decision Tree accuracy:  1.0


Теперь применим бэггинг:

In [8]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
bag_clf.fit(X_train, y_train)
print("Bagging accuracy: ", bag_clf.score(X_test, y_test))


Bagging accuracy:  1.0


C:\Users\Sidromnik\.conda\envs\TeachMeSkills\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


И наконец, применим бустинг:

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

boost_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
boost_clf.fit(X_train, y_train)
print("Boosting accuracy: ", boost_clf.score(X_test, y_test))


Boosting accuracy:  1.0


3. Сравнение результатов:

После выполнения кода каждый из студентов должен увидеть результаты точности для каждой из моделей. Это даст возможность сравнить эффективность каждого из методов на данной задаче классификации.

Важно отметить, что результаты могут варьироваться в зависимости от набора данных и параметров модели. Бустинг и бэггинг обычно обеспечивают лучшую обобщающую способность, чем одиночные деревья решений, но они также могут быть более подвержены переобучению, если не ограничивать сложность базовых моделей или не использовать регуляризацию.

При выполнении этого упражнения студенты могут экспериментировать с различными параметрами, такими как количество оценщиков (n_estimators) и коэффициент обучения (learning_rate), чтобы увидеть, как они влияют на результаты моделей.